<a href="https://colab.research.google.com/github/23117129/PYTORCH/blob/main/hmm_model_notebook1b4259f447.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/renewable-energy-and-weather-conditions/solar_weather.csv


In [ ]:
df = pd.read_csv('/kaggle/input/renewable-energy-and-weather-conditions/solar_weather.csv')
df.head()

,Time,Energy delta[Wh],GHI,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,isSun,sunlightTime,dayLength,SunlightTime/daylength,weather_type,hour,month
0,2017-01-01 00:00:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
1,2017-01-01 00:15:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
2,2017-01-01 00:30:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
3,2017-01-01 00:45:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
4,2017-01-01 01:00:00,0,0.0,1.7,1020,100,5.2,0.0,0.0,100,0,0,450,0.0,4,1,1


In [ ]:
weather_types= df['weather_type'].values
weather_types

array([4, 4, 4, ..., 3, 3, 3])

In [ ]:
weather_type_codes, unique_weather_types = pd.factorize(weather_types)
weather_type_codes , unique_weather_types

(array([0, 0, 0, ..., 2, 2, 2]), array([4, 5, 3, 2, 1]))

In [ ]:
n_states = len(unique_weather_types)
n_states


5

In [ ]:
pi = np.random.rand(n_states)
pi/=pi.sum()
pi

array([0.12296703, 0.28373925, 0.19790096, 0.13023339, 0.26515938])

In [ ]:
import numpy as np


weather_type_codes = [0, 1, 2, 0, 1, 1, 2, 0]  # Example data
n_states = 5  # Number of weather types

# Initialize the transition matrix A with zeros
A = np.zeros((n_states, n_states))

# Count transitions from one weather type to the next
for i in range(len(weather_type_codes) - 1):
    current_state = weather_type_codes[i]
    next_state = weather_type_codes[i + 1]
    A[current_state, next_state] += 1

# Compute the sum of each row
row_sums = A.sum(axis=1, keepdims=True)

# Normalize the transition matrix A
# Assign uniform probability for rows with zero sums
A = np.where(row_sums != 0, A / row_sums, 1.0 / n_states)

print("Transition Matrix A:")
print(A)



Transition Matrix A:
[[0.         1.         0.         0.         0.        ]
 [0.         0.33333333 0.66666667 0.         0.        ]
 [1.         0.         0.         0.         0.        ]
 [0.2        0.2        0.2        0.2        0.2       ]
 [0.2        0.2        0.2        0.2        0.2       ]]


/tmp/ipykernel_30/2222844460.py:22: RuntimeWarning: invalid value encountered in divide
  A = np.where(row_sums != 0, A / row_sums, 1.0 / n_states)


In [ ]:
B = np.eye(n_states)

print("Emission Matrix B:")
print(B)

Emission Matrix B:
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [ ]:
import numpy as np

def viterbi(observations, A, B, pi):

    n_states = A.shape[0]  # Number of hidden states
    T = len(observations)   # Length of observation sequence

    V = np.zeros((T, n_states))  # Viterbi table (log probabilities)
    path = np.zeros((T, n_states), dtype=int)  # Path table to store best states

    # Initialize the first column of Viterbi table (log initial probabilities and emissions)
    V[0, :] = np.log(pi) + np.log(B[:, observations[0]])

    for t in range(1, T):
        for s in range(n_states):
            # Calculate the transition probabilities from previous states
            transition_probs = V[t - 1] + np.log(A[:, s])
            best_prev_state = np.argmax(transition_probs)  # Find the best previous state
            V[t, s] = transition_probs[best_prev_state] + np.log(B[s, observations[t]])  # Update Viterbi table
            path[t, s] = best_prev_state  # Store the best previous state

    best_last_state = np.argmax(V[T - 1])
    best_path = [best_last_state]  # Initialize the best path

    for t in range(T - 1, 0, -1):
        best_last_state = path[t, best_last_state]
        best_path.insert(0, best_last_state)  # Insert it at the start of the path

    return best_path




# Run the Viterbi algorithm
best_hidden_states = viterbi(weather_type_codes, A, B, pi)
print("Most Likely Hidden State Sequence:", best_hidden_states)



Most Likely Hidden State Sequence: [0, 1, 2, 0, 1, 1, 2, 0]


/tmp/ipykernel_30/3190160551.py:13: RuntimeWarning: divide by zero encountered in log
  V[0, :] = np.log(pi) + np.log(B[:, observations[0]])
/tmp/ipykernel_30/3190160551.py:19: RuntimeWarning: divide by zero encountered in log
  transition_probs = V[t - 1] + np.log(A[:, s])  # V[t-1] + log(A)
/tmp/ipykernel_30/3190160551.py:21: RuntimeWarning: divide by zero encountered in log
  V[t, s] = transition_probs[best_prev_state] + np.log(B[s, observations[t]])  # Update Viterbi table


In [ ]:
 import numpy as np

def forward_backward(observations, A, B, pi):

    T = len(observations)  # Length of the observation sequence
    n_states = A.shape[0]  # Number of hidden states

    # Forward pass
    alpha = np.zeros((T, n_states))  # Initialize the forward probabilities
    alpha[0, :] = pi * B[:, observations[0]]  # Initialize first column

    # Calculate forward probabilities
    for t in range(1, T):
        for s in range(n_states):
            # Calculate alpha for state s at time t
            alpha[t, s] = B[s, observations[t]] * np.sum(alpha[t - 1, :] * A[:, s])

    # Backward pass
    beta = np.zeros((T, n_states))  # Initialize the backward probabilities
    beta[T - 1, :] = 1  # Initialize the last column

    # Calculate backward probabilities
    for t in range(T - 2, -1, -1):
        for s in range(n_states):
            # Calculate beta for state s at time t
            beta[t, s] = np.sum(beta[t + 1, :] * A[s, :] * B[:, observations[t + 1]])


    gamma = (alpha * beta) / np.sum(alpha * beta, axis=1, keepdims=True)

    return gamma


# Run the Forward-Backward algorithm
gamma = forward_backward(weather_type_codes, A, B, pi)
print("Posterior Probabilities (gamma):", gamma)


Posterior Probabilities (gamma): [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]]


In [ ]:
import numpy as np

def sample_hmm(A, B, pi, n_steps):

    n_states = A.shape[0]  # Number of hidden states
    states = []  # List to store sampled states
    observations = []  # List to store sampled observations

    # Sample initial state
    state = np.random.choice(n_states, p=pi)

    for _ in range(n_steps):
        states.append(state)  # Append the current state

        # Sample observation based on the current state
        observation = np.random.choice(len(B[0]), p=B[state, :])
        observations.append(observation)  # Append the observation

        # Sample next state
        state = np.random.choice(n_states, p=A[state, :])

    return states, observations


sampled_states, sampled_observations = sample_hmm(A, B, pi, 10)
print("Sampled Hidden States:", sampled_states)
print("Sampled Observations (Weather Types):", [weather_types[i] for i in sampled_observations])


Sampled Hidden States: [1, 2, 0, 1, 1, 2, 0, 1, 2, 0]
Sampled Observations (Weather Types): [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
